In [ ]:
# # Mecab 설치
# !apt-get update
# !apt-get install g++ openjdk-8-jdk 
# !pip3 install konlpy JPype1-py3
# !bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:2 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
g++ is already the newest version (4:9.3.0-1ubuntu2).
The following packages were automatically installed and are no longer required:
  accountsservice-ubuntu-schemas bc bluez-obexd cups cups-browsed cups-client
  cups-common cups-core-drivers cups-daemon cups-filters
  cups-filters-core-drivers cups-ipp-utils cups-ppdc cups-server-common
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript
  gir1.2-dbusmenu-glib-0.4 gnome-bluetooth gnome-power-manager
  gnome-screensaver gsettings-ubuntu-schemas gvfs-backends indicator-applet
  indicator-application indicator-appmenu indicator-bluetooth indicator-common

In [1]:
!pip install konlpy

In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from konlpy.tag import Okt, Mecab

# 데이터
- 전처리 및 맞춤법 검사가 된 문장

In [3]:
data = pd.read_csv('/aiffel/train_1000.csv')
data.head()

,Unnamed: 0,non_label_sentence,label_sentence,class,binary_class
0,0,아오 요즘 맨날 늦게 자고 일찍 일어나니까 피부가 꺼슬꺼슬 다크 서클 잔뜩 내려옴\...,1:아오 요즘 맨날 늦게 자고 일찍 일어나니까 피부가 꺼슬꺼슬 다크 서클 잔뜩 내려...,일반 대화,일반 대화
1,1,고양이를 키우는 건 정말 즐거운 일이야 감자를 캐자\n우리집 고양이는 보일러 틀어주...,1:고양이를 키우는 건 정말 즐거운 일이야 감자를 캐자\n2:우리집 고양이는 보일러...,일반 대화,일반 대화
2,2,조금 전에 뉴스 본 거 있어\n아니 뉴스에 무슨 기사 떴어\n완전 조금 전에 대박 ...,1:조금 전에 뉴스 본 거 있어\n2:아니 뉴스에 무슨 기사 떴어\n1:완전 조금 ...,일반 대화,일반 대화
3,3,난 강아지 같은 푸근한 인상이 좋아\n오 나는 원래 고양이 상 좋아했는데 나도 강아...,1:난 강아지 같은 푸근한 인상이 좋아\n2:오 나는 원래 고양이 상 좋아했는데 나...,일반 대화,일반 대화
4,4,니는 폰 게임 자주 하는 편이가\n가끔 삘 받으면 하긴 하는데 거의 안 하는 편 너...,1:니는 폰 게임 자주 하는 편이가\n2:가끔 삘 받으면 하긴 하는데 거의 안 하는...,일반 대화,일반 대화


In [125]:
# vocab_size = 30000
# sentences = data.non_label_sentence
# tokenizer = Tokenizer(num_words = vocab_size)
# tokenizer.fit_on_texts(sentences)
# word_dic = tokenizer.word_index
# sequences = tokenizer.texts_to_sequences(sentences)
# padded = pad_sequences(sequences)
# np.shape(padded)

(4913, 351)

In [231]:
okt = Okt()
vocab_size = 32000
sentences = data.non_label_sentence
sentences = [(okt.morphs(sen)) for sen in sentences]
tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(sentences)
word_dic = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(sentences)
padded = pad_sequences(sequences)
np.shape(padded)

(4913, 556)

In [232]:
len(word_dic)

32745

In [233]:
train_data = padded
train_label = data['class']
print(len(train_data), len(train_label))

4913 4913


In [234]:
labels = {'직장 내 괴롭힘 대화':0, '기타 괴롭힘 대화':1, '갈취 대화':2, '협박 대화':3, '일반 대화': 4}
train_label = train_label.apply(lambda x: labels[x])
train_label = pd.get_dummies(train_label)

from sklearn.model_selection import train_test_split

train_X, test_X, train_Y, test_Y = train_test_split(train_data, train_label, test_size=0.2, random_state=22)
valid_X, test_X, valid_Y, test_Y = train_test_split(test_X, test_Y, test_size=0.5, random_state=22)

print(len(train_X), len(valid_X), len(test_X))
print(len(train_Y), len(valid_Y), len(test_Y))

3930 491 492
3930 491 492


# 모델 
- LSTM 사용

In [235]:
word_vector_dim = 1024
labels_size = len(labels)
hidden_size = 128

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model.add(tf.keras.layers.LSTM(hidden_size, return_sequences=True))
model.add(tf.keras.layers.LSTM(hidden_size, return_sequences=True))
model.add(tf.keras.layers.LSTM(hidden_size//2))
model.add(tf.keras.layers.Dense(vocab_size, activation='relu'))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(labels_size, activation='softmax'))

model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, None, 1024)        32768000  
_________________________________________________________________
lstm_39 (LSTM)               (None, None, 128)         590336    
_________________________________________________________________
lstm_40 (LSTM)               (None, None, 128)         131584    
_________________________________________________________________
lstm_41 (LSTM)               (None, 64)                49408     
_________________________________________________________________
dense_26 (Dense)             (None, 32000)             2080000   
_________________________________________________________________
dropout_13 (Dropout)         (None, 32000)             0         
_________________________________________________________________
dense_27 (Dense)             (None, 5)               

In [236]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
              
epochs=10

history = model.fit(train_X,
                    train_Y,
                    epochs=epochs,
                    batch_size=256,
                    validation_data=(valid_X, valid_Y),
                    verbose=1)

Epoch 1/10
16/16 [==============================] - 10s 363ms/step - loss: 1.4240 - accuracy: 0.3415 - val_loss: 1.1030 - val_accuracy: 0.5234
Epoch 2/10
16/16 [==============================] - 5s 288ms/step - loss: 0.7115 - accuracy: 0.7359 - val_loss: 0.6183 - val_accuracy: 0.7760
Epoch 3/10
16/16 [==============================] - 5s 290ms/step - loss: 0.2648 - accuracy: 0.9097 - val_loss: 0.8316 - val_accuracy: 0.7312
Epoch 4/10
16/16 [==============================] - 5s 291ms/step - loss: 0.1009 - accuracy: 0.9672 - val_loss: 0.6167 - val_accuracy: 0.8289
Epoch 5/10
16/16 [==============================] - 5s 301ms/step - loss: 0.0152 - accuracy: 0.9962 - val_loss: 0.7659 - val_accuracy: 0.8411
Epoch 6/10
16/16 [==============================] - 5s 301ms/step - loss: 0.0236 - accuracy: 0.9921 - val_loss: 0.8206 - val_accuracy: 0.8086
Epoch 7/10
16/16 [==============================] - 5s 294ms/step - loss: 0.0091 - accuracy: 0.9967 - val_loss: 0.8356 - val_accuracy: 0.8167
Epoch

In [237]:
pred = model.predict(test_X)

from sklearn.metrics import classification_report

pred_label = []
test_label = []

for i in range(len(pred)):
  pred_label.append((str)(np.argmax(pred[i])))
  test_label.append((str)(labels[data['class'][test_Y.index[i]]]))

print(classification_report(test_label, pred_label))

              precision    recall  f1-score   support

           0       0.75      0.80      0.78       110
           1       0.68      0.73      0.70       115
           2       0.79      0.71      0.75        94
           3       0.75      0.70      0.72        69
           4       0.97      0.95      0.96       104

    accuracy                           0.78       492
   macro avg       0.79      0.78      0.78       492
weighted avg       0.79      0.78      0.79       492



In [238]:
to_text = {0:'직장 내 괴롭힘 대화', 1:'기타 괴롭힘 대화', 2:'갈취 대화', 3:'협박 대화', 4:'일반 대화'}
result = pd.DataFrame([pred_label,test_label]).T
error_target = result[result[0] != result[1]].reset_index(drop=True)
idx = result[result[0] != result[1]].index
error = data.non_label_sentence[idx]
idx = pd.DataFrame(zip(idx))
error = pd.DataFrame(zip(error))
error_result = pd.concat([idx,error,error_target], axis=1)
error_result.columns = ['idx', 'conversation', 'pred', 'real']
error_result['pred'] = error_result['pred'].apply(lambda x: to_text[int(x)])
error_result['real'] = error_result['real'].apply(lambda x: to_text[int(x)])

In [239]:
Sam = error_result[error_result['real'] == '직장 내 괴롭힘 대화'].sample(n=2)
id = Sam.index
Sam[['idx','pred', 'real']]

,idx,pred,real
2,8,기타 괴롭힘 대화,직장 내 괴롭힘 대화
104,481,기타 괴롭힘 대화,직장 내 괴롭힘 대화


In [240]:
Sam['conversation'][id[0]]

'우리 강아지 오늘 산책을 못했어\n아니 저번에 보내준다던 내 반려 문어는 어떻게 거\n하루 종일 비가 와가꾸\n벅수 산책을 못 갔다고 줘런\n아 반려 문어 돌잔치 때문에 못 잡았어\n강아지는 산책이 최고인데\n내말이 강아지 산책이 힐링인디\n줴가 문어 잡아서 보내준댔거든 반려문어\n강아지 목욕도 시켜야 하는데\n반려 갑오징어도 한 마리 부탁해\n강아지 목욕 시키지 진짜 힘듬\n어 오늘 날씨 정말 춥더라\n알았어 내가 바로 어부닷\n강아지 목욕은 그 드라이룸만 있으면 좀 수월할 거 같아\n어 그거 즘 사주라 로또 되면\n먹을 거야 아마도\n로또 되고싶다'

In [241]:
Sam['conversation'][id[1]]

'는 어짜피 군대 가야지\n군대는 사람을 힘들게 하는 곳이지\n가서 인내심을 배워 그럼 농사 할 수 있어\n가서 잘하면 선임들한테 칭찬을 받지\n그건 아닌데 농사 하려면 인내심은 기본이야\n인내심은 모든 곳의 기본이지\n하하 오빠는 군대에서 인내심을 길렀어\n군대에 가면 자기개발은 무었을 할 거니\n오빠는 후임들 괴롭히고 놀았을 듯\n난 족구를 열심히 하며 시간을 가졌어\n오빠 축구 잘 못하지 않어\n족구 마스터가 되어 나오게 될 거야 너는\n난 축구 족구 피구 다 못해\n난 북일고에서 축구로 제일가는 선수였어\n난 배드민턴 잘하는데 군대에서 이것도 좋아해줘\n축구는 잘 하는 편이니\n군대는 풋살 못하면 친해지기 힘들 것이야'

In [242]:
Sam = error_result[error_result['real'] == '기타 괴롭힘 대화'].sample(n=2)
id = Sam.index
Sam[['idx','pred', 'real']]

,idx,pred,real
8,35,갈취 대화,기타 괴롭힘 대화
42,198,일반 대화,기타 괴롭힘 대화


In [243]:
Sam['conversation'][id[0]]

'니는 아직 결혼 생각 없는 거가\n어 나는 결혼 천천히 할 생각이야 너는\n주변 친구들 결혼 많이 하지 않았나\n주변에 나름 좀 하긴 했지\n근데 친한 친구들은 그렇게 많이 한 편은 아니라서 하하\n나도 결혼은 미룰 수 있을 때 까지 미뤄야 하지 않겠나\n맞는 말이다 우리 평균 수명이 늘어나서 천천히 가야 된다\n나도 친한 친구들 중에 결혼 한 애가 한 명 밖에 없어 아직\n한 명 진짜 많이 없네\n결혼까지는 좋은데 애 낳고 하면 힘들다\n그래 그래 결혼식 안 간지도 엄청 오래됐다 나는\n나는 올해 6월에 절친 결혼해서 갔다 왔어\n그렇지 요즘 아무리 남편이랑 같이 육아하고 한다 해도 여자들이 많이 손해를 보는 것 같다 아직까진\n어 대부분의 남편들 육아 잘 안 함\n난 내년에 친구 하나 결혼 예정이라 결혼식 가 볼 수 있다\n그 전에 코로나가 좀 나아져야 할 텐데\n어어 육아 같이 한다고 말은 하던데\n실제로 잘 안되겠지\n애를 또 잘 보는 남자가 잘 없더라'

In [244]:
Sam['conversation'][id[1]]

'이번 여름 장마에는 비가 많이 안 왔어요\n그러게요 여름에 안 오고 오히려 가을에 많이 온 것 같은데요\n이번 여름에 하도 무더워서 고생했는데 금방 가을이 온 것 같아요\n맞아요 여름에 우리 셋이 강아지들 산책 시키느라 땀 뻘뻘 흘렸죠\n그러게요 우리 날도 좋아졌으니 어디 놀러 갈까요\n너무 좋죠 그런데 저 어릴 때는 추석 때 추워서 두꺼운 스웨터 입었던 기억이 나는데 날씨가 많이 변한 것 같아요\n전 그나마 가을 도 좋고 봄도 괜찮은데 두 분은 어떤 계절 좋아하세요\n전 봄이랑 가을요\n근데 가을이 더 좋은 것 같아요\n봄에는 미세먼지가 좀 있잖아요\n아 맞아요 저도 미세먼지 정말 싫어요\n그런데 전 여름도 좋고 겨울도 좋아요\n사실 여러 스포츠를 즐길 수 있어서요\n요가도 잘하시던데 다른 운동도 좋아하나 봐요\n네 저는 계절마다 할 수 있는 것들을 다 해보려고 하는 편이에요\n와 대단하네요 저는 사실 집에 있는 걸 제일 좋아해요 집순이에요\n저는 힘들게 하는 걸 더 좋아해요\n가을에는 설악산이나 지리산에 가서 등산하는 게 최고죠\n헐 저는 가을에 그냥 집 근처 산책하는 게 더 좋아요 아니면 그냥 가까운 곳이요\n전 가을에 산에 가서 단풍 보면 행복해지더라구요\n그럼 우리 가을에 같이 산에 단풍 보러 갈까요 강아지들 데리구요'

In [245]:
Sam = error_result[error_result['real'] == '갈취 대화'].sample(n=2)
id = Sam.index
Sam[['idx','pred', 'real']]

,idx,pred,real
3,11,기타 괴롭힘 대화,갈취 대화
18,98,기타 괴롭힘 대화,갈취 대화


In [246]:
Sam['conversation'][id[0]]

'니는 결혼식 크게 할 생각이가\n어 한 번 하는 건데 크게 해야지\n니는 외동이라서 진짜 크게 해야 된다\n니는 크게 안 할 거가\n나는 첫째라서 집안에서 크게 하라고 그러더라\n그래 한 번 하는 건데 기억에 남게 해야지\n결혼할 때도 돈 진짜 많이 들어갈 텐데\n결혼식은 축의금으로 거의 계산한대\n축의금으로 그게 맞춰지는가\n생각보다 많이 들어온다던데\n근데 우리는 다 부모님이 뿌린 만큼\n부모님 지인이지 뭐 ㅋ\n맞다 받으면 돌려줘야지\n결혼해서는 이제 우리가 뿌려야 된다\n맞다 니 친구들은 결혼 다 했나\n아니 결혼한 애들도 있는데 안 한 애들이 더 많다\n내 주위도 안 한 사람이 많타'

In [247]:
Sam['conversation'][id[1]]

'여전히 보험금 사기 사건은 끊이질 않네\n나도 봤어 기사 고액 알바로 공범 모집해서 보험금 사기 친 거\n근데 주범이 되게 대담한 것 같아\n고액 알바 구한다고 구인구직 사이트에서 공범을 모집하다니\n그러니까 그런 쪽으로 잘 아는 건가\n정확히 어떤 거 보험금 사기를 치는 거야\n미리 짜고 교통사고를 낸 다음에 보험사로부터 합의금이랑 차량 수리비를 받는 거지\n보험이 그러라고 있는 게 아닌데\n그런 보험금 사기 사건들 때문에 일반인들만 힘들어지지\n진짜 교통사고 나면 보험금 받기 너무 까다롭다고 하더라\n어쨌든 보험도 확인을 하고 보험금을 지급해야 하니까\n그렇긴 하지\n보험금 사기 사건이 한둘이 아니니까\n진짜 보험금 사기 사건이 여러 방법으로 일어나서 경찰이나 보험나 힘들 것 같아\n나도 그럴 것 같다는 생각했어\n보험금 사기 사건으로 얼마 전에도 엄청 이슈였잖아\n아 외제차값 갚으려고 보험금 노리고 여자친구 살해하려다 살인 미수된 사건'

In [248]:
Sam = error_result[error_result['real'] == '협박 대화'].sample(n=2)
id = Sam.index
Sam[['idx','pred', 'real']]

,idx,pred,real
11,47,갈취 대화,협박 대화
90,390,기타 괴롭힘 대화,협박 대화


In [249]:
Sam['conversation'][id[0]]

'너는 아르바이트 많이 해봤어\n아뇨 하하\n가끔 사촌형 가게 알바 했어요\n아 진짜\n어떤 가게 업무였어\n사촌형이 떡집을 해서 힘 쓰는 일이 많이 필요해요\n아 진짜\n떡집은 바쁜 시즌이 딱 있는 거지\n넵 하하 아무래도 명절이 대목이죠\n명절 때 진짜 바빠서 업무가 많겠다\n명절 2주 전부터 떡집은 전쟁이죠 하하\n그럼 떡 엄청 많이 먹을 수 있었겠네\n넵 하하 근데 떡을 좋아하지는 않아서 그냥 일만 도왔어요\n뭔가 알바할 때 급여가 높았을 거 같은데\n그렇게 많이 받지는 못했어요 하하\n사촌 형이 급여 많이 챙겨줘서 좋았겠다\n많이 챙겨줬으면 좋았을텐데 아쉽네요 하하\n그래도 뭔가 부수입이 있으면 좋지 않나\n넵 하하 그런 점에서는 좋아요 하하'

In [250]:
Sam['conversation'][id[1]]

'다이어트는 평생의 숙제인 듯해\n그치 나도 올해 연말까지 몇 키로 만들기로 했는데\n운동은 그렇다 쳐 식이조절 진짜 너무 하기 싫어\n난 둘 다 식이가 더 힘들긴 해\n아까 다이어트 도시락 얘기했잖아\n먹고 있어\n아 작년에 좀 먹다가 때려쳤지\n저녁에 더 많이 먹게 되어서\n하나에 330칼로리쯤 하는데 두 개 먹어야 대\n그러니까 맛있어서 많이 먹게 되더라고\n나 요즘 투고샐러드에 빠짐\n맛있어\n나는 사 먹는 샐러드는 서브웨이밖에 안 먹어봤어\n웅 맛나는데 문제는 샐러드가 맛나서 먹는 게 아니라 계란밥을 먹어 목살 얹어서\n반찬이네 샐러드는\n근데 샐러드도 넘 맛있게 잘 나오더라 요즘\n맞아 파스타 샐러드 특히 바질\n와 말만 들어도 맛있겠다\n그래서 다이어트 떄문에 먹는 게 맞는 건지 싶다니까\n나는 점심때 거의 나가서 외식 하는데 항상 고칼로리만 먹어서 걱정이야'

In [251]:
Sam = error_result[error_result['real'] == '일반 대화'].sample(n=2)
id = Sam.index
Sam[['idx','pred', 'real']]

,idx,pred,real
47,228,직장 내 괴롭힘 대화,일반 대화
16,76,직장 내 괴롭힘 대화,일반 대화


In [252]:
Sam['conversation'][id[0]]

'야 너네 다음 카페 업뎃 된 거 들었어\n아니 다음 카페를 안 해서 몰긋다\n아니 그게 악성 페미니즘 카페가 다음에 있거든\n아 어어 무서운 카페 그거\n어어 우리가 지겹게 시달린 그곳이 다음 카페에 있단 말이야\n어 근데 머가 업뎃 됐는디\n응응 거기에 원래 글도 익명 댓글도 익명으로 올렸었나 봐\n근데 이번에 업뎃 해 가지고 글쓴이가 쓴 댓글은 익명이어도 옆에 글쓴이 댓글이라고 뱃지가 달려\n그럼 머 악플 이런 게 좀 덜 해지낭\n아니 그건 아닌데\n악성 페미 카페에 망상글 올리고 댓글로 싸우고 남자 욕하고 사회 욕하고 글 올린 사람 옹호하고 이런 댓글이 엄청 많았을 거 아니야\n어엉 그런 거 장난 아니었지\n근데 그게 알고 보니까 다 글쓴이 뱃지가 달린 댓글인 거야\n한마디로 자기가 글 쓰고 자기가 주작질 해서 댓글 쓴 거지\n아아 헐 대체 왜 그러는 거여 관심 받고 싶나\n심지어 그런 글이 한두 개가 아니라 거진 대다수인 게 밝혀져서 여론 장난 없음 진짜\n에혀 네이버도 연옌들 악플 심해서 댓글창 없애고 그랬었자너\n맞아 진짜 주작들이나 망상 글은 왜 이렇게 해대는지 이해가 안 가 난\n마자 다들 피해 의식이 장난 아닌 듯'

In [253]:
Sam['conversation'][id[1]]

'저녁 메뉴로 소고기 먹으려고\n저녁 메뉴가 소고기라니 부럽다\n저녁이니까 메뉴가 과해야지\n갑자기 소고기가 땡겼어\n살치살도 맛있고 부채살도 맛있고\n새우살은 한 번도 안 먹어 봤는데 먹어봐야겠네\n나도 새우살은 안 먹어 봤어\n하 부럽다\n그렇지 소고기는 다 맛있지\n등심 안심 소고기는 사랑입니다\n맞다 육회랑 육사시미 다 먹어 주겠어\n누구랑 가는데 저녁 메뉴가 어마어마해\n부모님이랑 밖에 나가서 먹고 오려고\n아 그럼 먹어야지\n부모님 좋아하시겠다\n응 엄마가 소고기 먹고 싶다고 하셔서\n그럼 풀로 다 먹어 줘야지\n세 명이 먹는데 너무 많지 않을까\n다 1인분씩 시켜서 먹으면 되지'